In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler
from xgboost.sklearn import XGBRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor

import time
import os
import random
from collections import Counter

pd.set_option('display.max_columns', None)

df = pd.read_csv('data/대곡/2012 - 복사본.csv', encoding='cp949')

df.drop(columns='강수량', inplace=True)
df

C:\Users\leedo\miniconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,일시,50CM 정시 토양수분(%),0.5M 정시 습도(%),1.5M 정시 습도(%),4.0M 정시 습도(%),10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),0.5M 정시 기온(°C),1.5M 정시 기온(°C),4.0M 정시 기온(°C),1.5M 정시 풍속(m/s),4.0M 정시 풍속(m/s),정시 지면온도(°C),5CM 정시 지중온도(°C),10CM 정시 지중온도(°C),20CM 정시 지중온도(°C),30CM 정시 지중온도(°C),정시 누적 순복사(MJ/m2),정시 누적 전천복사(MJ/m2),정시 누적 반사복사(MJ/m2)
0,2012-01-01 0:00,10.0,71.1,70.8,66.6,16.9,11.7,6.4,-3.1,-2.9,-2.6,0.3,0.3,-3.1,0.0,0.5,1.7,2.6,7.12,10.37,3.25
1,2012-01-01 1:00,10.0,76.0,76.2,71.3,17.0,11.7,6.4,-4.2,-3.8,-3.5,0.3,0.5,-3.9,0.0,0.5,1.6,2.6,0.00,0.00,0.00
2,2012-01-01 2:00,10.0,78.5,78.4,74.2,17.0,11.7,6.4,-4.9,-4.6,-4.2,0.4,0.3,-4.4,-0.1,0.4,1.6,2.6,0.00,0.00,0.00
3,2012-01-01 3:00,10.0,81.8,81.8,78.8,17.0,11.7,6.4,-5.5,-5.3,-5.1,0.0,0.0,-4.7,-0.2,0.3,1.6,2.6,0.00,0.00,0.00
4,2012-01-01 4:00,10.0,83.3,83.3,84.4,16.9,11.7,6.4,-5.8,-5.5,-5.2,0.1,0.0,-4.9,-0.3,0.3,1.6,2.6,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455875,2017-12-31 19:00,29.2,58.0,51.2,53.3,5.9,25.8,25.6,2.6,2.7,2.7,2.2,3.2,2.5,4.1,4.4,4.6,4.6,4.16,6.65,2.49
455876,2017-12-31 20:00,29.2,65.3,58.1,58.8,5.9,25.8,25.6,0.4,0.6,0.6,0.4,0.6,1.7,3.7,4.1,4.5,4.6,4.16,6.65,2.49
455877,2017-12-31 21:00,29.2,78.0,65.8,65.2,5.9,25.8,25.6,-1.1,-0.6,-0.3,0.4,0.6,1.3,3.3,3.9,4.4,4.6,4.16,6.65,2.49
455878,2017-12-31 22:00,29.2,82.5,73.0,74.5,5.9,25.8,25.6,-1.7,-1.5,-1.5,0.2,0.1,1.0,2.9,3.6,4.2,4.6,4.16,6.65,2.49


In [2]:
df.shape

(455880, 21)

In [3]:
df.head

<bound method NDFrame.head of                       일시  50CM 정시 토양수분(%)  0.5M 정시 습도(%)  1.5M 정시 습도(%)  \
0        2012-01-01 0:00             10.0           71.1           70.8   
1        2012-01-01 1:00             10.0           76.0           76.2   
2        2012-01-01 2:00             10.0           78.5           78.4   
3        2012-01-01 3:00             10.0           81.8           81.8   
4        2012-01-01 4:00             10.0           83.3           83.3   
...                  ...              ...            ...            ...   
455875  2017-12-31 19:00             29.2           58.0           51.2   
455876  2017-12-31 20:00             29.2           65.3           58.1   
455877  2017-12-31 21:00             29.2           78.0           65.8   
455878  2017-12-31 22:00             29.2           82.5           73.0   
455879  2017-12-31 23:00             29.2           86.8           81.0   

        4.0M 정시 습도(%)  10CM 정시 토양수분(%)  20CM 정시 토양수분(%)  30CM 정시 토양수분

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455880 entries, 0 to 455879
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   일시                 455880 non-null  object 
 1   50CM 정시 토양수분(%)    455880 non-null  float64
 2   0.5M 정시 습도(%)      455880 non-null  float64
 3   1.5M 정시 습도(%)      455880 non-null  float64
 4   4.0M 정시 습도(%)      455880 non-null  float64
 5   10CM 정시 토양수분(%)    455880 non-null  float64
 6   20CM 정시 토양수분(%)    455880 non-null  float64
 7   30CM 정시 토양수분(%)    455880 non-null  float64
 8   0.5M 정시 기온(°C)     455880 non-null  float64
 9   1.5M 정시 기온(°C)     455880 non-null  float64
 10  4.0M 정시 기온(°C)     455880 non-null  float64
 11  1.5M 정시 풍속(m/s)    455880 non-null  float64
 12  4.0M 정시 풍속(m/s)    455880 non-null  float64
 13  정시 지면온도(°C)        455880 non-null  float64
 14  5CM 정시 지중온도(°C)    455880 non-null  float64
 15  10CM 정시 지중온도(°C)   455880 non-null  float64
 16  20

In [5]:
df.describe()

,50CM 정시 토양수분(%),0.5M 정시 습도(%),1.5M 정시 습도(%),4.0M 정시 습도(%),10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),0.5M 정시 기온(°C),1.5M 정시 기온(°C),4.0M 정시 기온(°C),1.5M 정시 풍속(m/s),4.0M 정시 풍속(m/s),정시 지면온도(°C),5CM 정시 지중온도(°C),10CM 정시 지중온도(°C),20CM 정시 지중온도(°C),30CM 정시 지중온도(°C),정시 누적 순복사(MJ/m2),정시 누적 전천복사(MJ/m2),정시 누적 반사복사(MJ/m2)
count,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000,455880.000000
mean,20.572760,71.510287,72.382669,70.607706,14.248341,18.586302,18.623464,12.544971,12.535596,12.671050,1.164623,1.484409,14.580713,14.485183,14.514917,14.571970,14.643910,6.026931,7.929508,1.981024
std,10.603455,22.679753,22.348974,22.742597,8.658342,8.298075,9.046627,10.971359,10.785453,10.810267,1.132448,1.322998,12.455266,10.456532,10.122399,9.658449,9.272915,6.864941,8.628061,2.271256
min,0.000000,1.800000,0.000000,-999.000000,0.000000,0.000000,0.000000,-39.400000,-27.800000,-26.300000,0.000000,0.000000,-16.900000,-36.500000,-10.000000,-5.300000,-4.200000,-6.680000,0.000000,0.000000
25%,12.300000,54.500000,55.900000,53.900000,7.400000,11.900000,11.600000,3.500000,3.700000,3.800000,0.300000,0.500000,3.400000,4.700000,5.000000,5.400000,5.900000,0.110000,0.200000,0.070000
50%,19.200000,75.800000,76.100000,73.800000,14.300000,17.600000,18.000000,13.400000,13.400000,13.500000,0.900000,1.100000,14.700000,15.000000,15.100000,15.100000,15.200000,3.570000,5.000000,1.200000
75%,26.700000,91.800000,92.600000,90.900000,20.100000,24.400000,25.700000,21.800000,21.700000,21.800000,1.700000,2.100000,23.800000,23.700000,23.700000,23.500000,23.400000,10.250000,13.410000,3.210000
max,98.000000,99.900000,99.900000,99.900000,53.600000,56.400000,99.900000,55.200000,55.900000,60.000000,14.000000,14.300000,65.500000,49.700000,43.700000,49.500000,52.300000,52.630000,52.620000,15.000000


In [6]:
df_col = df.columns[1:]
df_col

Index(['50CM 정시 토양수분(%)', '0.5M 정시 습도(%)', '1.5M 정시 습도(%)', '4.0M 정시 습도(%)',
       '10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',
       '0.5M 정시 기온(°C)', '1.5M 정시 기온(°C)', '4.0M 정시 기온(°C)', '1.5M 정시 풍속(m/s)',
       '4.0M 정시 풍속(m/s)', '정시 지면온도(°C)', '5CM 정시 지중온도(°C)', '10CM 정시 지중온도(°C)',
       '20CM 정시 지중온도(°C)', '30CM 정시 지중온도(°C)', '정시 누적 순복사(MJ/m2)',
       '정시 누적 전천복사(MJ/m2)', '정시 누적 반사복사(MJ/m2)'],
      dtype='object')

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df.iloc[:, 1:])
df_scaled = scaler.transform(df.iloc[:, 1:])

x, y = df_scaled[:,1:], df_scaled[:,0]

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=111)

In [24]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)

In [25]:
print("x_train Data shape: {}".format(x_train.shape), "y_train Data shape: {}".format(y_train.shape))
print("x_val Data shape: {}".format(x_val.shape), "y_val Data shape: {}".format(y_val.shape))
print("x_test Data shape: {}".format(x_test.shape), "y_test Data shape: {}".format(y_test.shape))

x_train Data shape: (307719, 19) y_train Data shape: (307719,)
x_val Data shape: (102573, 19) y_val Data shape: (102573,)
x_test Data shape: (45588, 19) y_test Data shape: (45588,)


## 랜덤포레스트모델

In [107]:
#시간 체크용
start = time.time()     

rf= RandomForestRegressor(n_jobs=-1, random_state=111)

param_grid = {
    'n_estimators':[50, 100, 150, 200, 400],
    'max_depth':list(range(5, 20)),
    'min_samples_leaf':list(range(2, 5)),
    'min_samples_split':list(range(1, 5)),
    'max_samples': [0.2, 0.4, 0.7, 0.8, 0.9, 1.0],
    'max_features': [0.2, 0.4, 0.7, 0.8, 0.9, 1.0]
    }

# ss=ShuffleSplit(test_size=0.3,random_state=111)
grid_rf_cv=RandomizedSearchCV(rf, 
                              param_distributions=param_grid, 
                              cv=5, 
                             verbose=1,
                             n_jobs=-1, 
                             random_state=111)

grid_rf_cv.fit(x_train,y_train)

end=time.time()
print('수행 시간: {0:.3f}'.format(end - start))
print('최적의 매개변수 조합: ', grid_rf_cv.best_params_)
print('최고의 교차 검증 점수: ', grid_rf_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\leedo\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\leedo\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\leedo\miniconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\leedo\miniconda3\lib\site-packages\joblib\parallel.py", line 1098, in __call__
    self.retrieve()
  File "C:\Users\leedo\miniconda3\lib\site-packages\joblib\parallel.py"

수행 시간: 697.734
최적의 매개변수 조합:  {'n_estimators': 100, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_samples': 0.9, 'max_features': 0.8, 'max_depth': 19}
최고의 교차 검증 점수:  0.9727374017070719


In [108]:
rf = RandomForestRegressor(n_estimators=grid_rf_cv.best_params_['n_estimators'] , 
                          max_depth=grid_rf_cv.best_params_['max_depth'] ,
                          min_samples_leaf=grid_rf_cv.best_params_['min_samples_leaf'] ,
                          min_samples_split=grid_rf_cv.best_params_['min_samples_split'] ,
                           max_samples=grid_rf_cv.best_params_['max_samples'] ,
                           max_features=grid_rf_cv.best_params_['max_features'] ,
                          n_jobs = -1, 
                           random_state = 111)
rf.fit(x_train, y_train)

RandomForestRegressor(max_depth=19, max_features=0.8, max_samples=0.9,
                      min_samples_leaf=3, min_samples_split=4, n_jobs=-1,
                      random_state=111)

In [168]:
pred_train = rf.predict(x_train)
pred_val = rf.predict(x_val)
pred_test = rf.predict(x_test)

In [169]:
# train set의 예측결과
rf_train_mae = mean_absolute_error(y_train, pred_train)
rf_train_mse = mean_squared_error(y_train, pred_train)
rf_train_rmse = np.sqrt(rf_train_mse)
rf_train_r2 = r2_score(y_train, pred_train)
print('MAE : ', round(rf_train_mae, 3))
print('MSE : ', round(rf_train_mse, 3))
print('RMSE :', round(rf_train_rmse, 3))
print('r2 score :', round(rf_train_r2, 3))

MAE :  0.044
MSE :  0.011
RMSE : 0.106
r2 score : 0.989


In [170]:
# val set의 예측결과
rf_val_mae = mean_absolute_error(y_val, pred_val)
rf_val_mse = mean_squared_error(y_val, pred_val)
rf_val_rmse = np.sqrt(rf_val_mse)
rf_val_r2 = r2_score(y_val, pred_val)
print('MAE : ', round(rf_val_mae, 3))
print('MSE : ', round(rf_val_mse, 3))
print('RMSE :', round(rf_val_rmse, 3))
print('r2 score :', round(rf_val_r2, 3))

MAE :  0.059
MSE :  0.023
RMSE : 0.151
r2 score : 0.977


In [171]:
# 최종 test set의 결과
rf_mae = mean_absolute_error(pred_test, y_test)
rf_mse = mean_squared_error(pred_test, y_test)
rf_rmse = np.sqrt(rf_mse)
rf_r2 = r2_score(pred_test, y_test)
print('MAE : ', round(rf_mae, 3))
print('MSE : ', round(rf_mse, 3))
print('RMSE :', round(rf_rmse, 3))
print('r2 score :', round(rf_r2, 3))

MAE :  0.058
MSE :  0.022
RMSE : 0.148
r2 score : 0.976


## XGB모델

In [37]:
#시간 체크용
start = time.time()     

reg = XGBRegressor(n_jobs=-1, random_state=111)

param_grid = {
    'n_estimators':[50, 100, 150, 200, 400],
    'max_depth':list(range(5, 20)),
    'gamma': [0.01, 0.02, 0.03],
    'importance_type': ['weight', 'gain', 'cover', 'total_gain', 'total_cover'],
    'reg_lambda': [0.1, 0.2, 0.3, 0.4, 0.7, 0.8, 0.9, 1.0]
    }

# ss=ShuffleSplit(test_size=0.3,random_state=111)
grid_rf_cv=RandomizedSearchCV(reg, 
                              param_distributions=param_grid, 
                              cv=5, 
                             verbose=1,
                             n_jobs=-1, 
                             random_state=111)

grid_rf_cv.fit(x_train,y_train)

end=time.time()
print('수행 시간: {0:.3f}'.format(end - start))
print('최적의 매개변수 조합: ', grid_rf_cv.best_params_)
print('최고의 교차 검증 점수: ', grid_rf_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
수행 시간: 3070.176
최적의 매개변수 조합:  {'reg_lambda': 0.7, 'n_estimators': 200, 'max_depth': 19, 'importance_type': 'total_gain', 'gamma': 0.02}
최고의 교차 검증 점수:  0.9767776267225207


In [38]:
reg = XGBRegressor(n_estimators=grid_rf_cv.best_params_['n_estimators'] , 
                  max_depth=grid_rf_cv.best_params_['max_depth'] ,
                  gamma=grid_rf_cv.best_params_['gamma'] ,
                  importance_type=grid_rf_cv.best_params_['importance_type'] ,
                   reg_lambda=grid_rf_cv.best_params_['reg_lambda'] ,
                  n_jobs = -1, 
                   random_state = 111)

reg.fit(x_train, y_train)

In [172]:
pred_train = reg.predict(x_train)
pred_val = reg.predict(x_val)
pred_test = reg.predict(x_test)

In [173]:
# train set의 예측결과
reg_train_mae = mean_absolute_error(y_train, pred_train)
reg_train_mse = mean_squared_error(y_train, pred_train)
reg_train_rmse = np.sqrt(reg_train_mse)
reg_train_r2 = r2_score(y_train, pred_train)
print('MAE : ', round(reg_train_mae, 3))
print('MSE : ', round(reg_train_mse, 3))
print('RMSE :', round(reg_train_rmse, 3))
print('r2 score :', round(reg_train_r2, 3))

MAE :  0.019
MSE :  0.001
RMSE : 0.028
r2 score : 0.999


In [174]:
# val set의 예측결과
reg_val_mae = mean_absolute_error(y_val, pred_val)
reg_val_mse = mean_squared_error(y_val, pred_val)
reg_val_rmse = np.sqrt(reg_val_mse)
reg_val_r2 = r2_score(y_val, pred_val)
print('MAE : ', round(reg_val_mae, 3))
print('MSE : ', round(reg_val_mse, 3))
print('RMSE :', round(reg_val_rmse, 3))
print('r2 score :', round(reg_val_r2, 3))

MAE :  0.044
MSE :  0.02
RMSE : 0.14
r2 score : 0.98


In [175]:
# 최종 test set의 결과
reg_mae = mean_absolute_error(pred_test, y_test)
reg_mse = mean_squared_error(pred_test, y_test)
reg_rmse = np.sqrt(reg_mse)
reg_r2 = r2_score(pred_test, y_test)
print('MAE : ', round(reg_mae, 3))
print('MSE : ', round(reg_mse, 3))
print('RMSE :', round(reg_rmse, 3))
print('r2 score :', round(reg_r2, 3))

MAE :  0.043
MSE :  0.017
RMSE : 0.131
r2 score : 0.982


## SVR모델 실패사유: 너무 오래걸림

In [12]:
sv_regressor = SVR(kernel='rbf',
                   degree=4,
                   gamma='scale',
                   coef0=0.0,
                   tol=0.001,
                   C=1.0,
                   epsilon=0.1,
                   shrinking=True,
                   cache_size=200,
                   verbose=False,
                   max_iter=-1)

In [ ]:
sv_regressor.fit(x_train,y_train)

pred_train = sv_regressor.predict(x_train)
pred_val = sv_regressor.predict(x_val)
y_pred =sv_regressor.predict(x_test)

In [ ]:
mae = mean_absolute_error(pred_test, y_test)
mse = mean_squared_error(pred_test, y_test)
rmse = np.sqrt(mse)
r2 = r2_score(pred_test, y_test)
print('MAE : ', round(mae, 3))
print('MSE : ', round(mse, 3))
print('RMSE :', round(rmse, 3))
print('r2 score :', round(r2, 3))

## LGBM모델

In [52]:
#시간 체크용
start = time.time()     

lgbm = LGBMRegressor(n_jobs=-1, random_state=111)

param_grid = {
    'n_estimators':[50, 100, 150, 200, 400],
    'num_leaves':list(range(30, 500, 10)),
    'min_split_gain': [0.0, 0.01, 0.02, 0.03, 0.04, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.2, 0.3, 0.4],
    'min_child_weight':[0.0, 0.001, 0.002, 0.003, 0.004, 0.01, 0.02, 0.03, 0.04, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.2, 0.3, 0.4],
    'min_child_samples':list(range(20, 50)),
    'learning_rate':[0.1, 0.2, 0.3, 0.4, 0.5],
    'importance_type': ['gbdt', 'dart', 'goss', 'rf'],
    'reg_lambda': [0.1, 0.2, 0.3, 0.4, 0.7, 0.8, 0.9, 1.0]
    }

# ss=ShuffleSplit(test_size=0.3,random_state=111)
grid_rf_cv=RandomizedSearchCV(lgbm, 
                              param_distributions=param_grid, 
                              cv=5, 
                             verbose=1,
                             n_jobs=-1, 
                             random_state=111)

grid_rf_cv.fit(x_train,y_train)

end=time.time()
print('수행 시간: {0:.3f}'.format(end - start))
print('최적의 매개변수 조합: ', grid_rf_cv.best_params_)
print('최고의 교차 검증 점수: ', grid_rf_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
수행 시간: 62.696
최적의 매개변수 조합:  {'reg_lambda': 0.7, 'num_leaves': 400, 'n_estimators': 400, 'min_split_gain': 0.01, 'min_child_weight': 0.1, 'min_child_samples': 39, 'learning_rate': 0.1, 'importance_type': 'rf'}
최고의 교차 검증 점수:  0.9714992770960851


In [53]:
lgbm = LGBMRegressor(n_estimators=grid_rf_cv.best_params_['n_estimators'] , 
                  num_leaves=grid_rf_cv.best_params_['num_leaves'] ,
                  min_split_gain=grid_rf_cv.best_params_['min_split_gain'] ,
                   min_child_weight=grid_rf_cv.best_params_['min_child_weight'] ,
                     min_child_samples=grid_rf_cv.best_params_['min_child_samples'] ,
                     learning_rate=grid_rf_cv.best_params_['learning_rate'] ,
                     importance_type=grid_rf_cv.best_params_['importance_type'] ,
                     reg_lambda=grid_rf_cv.best_params_['reg_lambda'] ,
                  n_jobs = -1, 
                   random_state = 111)

lgbm.fit(x_train, y_train)

In [176]:
pred_train = lgbm.predict(x_train)
pred_val = lgbm.predict(x_val)
pred_test = lgbm.predict(x_test)

In [177]:
# train set의 예측결과
lgbm_train_mae = mean_absolute_error(y_train, pred_train)
lgbm_train_mse = mean_squared_error(y_train, pred_train)
lgbm_train_rmse = np.sqrt(lgbm_train_mse)
lgbm_train_r2 = r2_score(y_train, pred_train)
print('MAE : ', round(lgbm_train_mae, 3))
print('MSE : ', round(lgbm_train_mse, 3))
print('RMSE :', round(lgbm_train_rmse, 3))
print('r2 score :', round(lgbm_train_r2, 3))

MAE :  0.057
MSE :  0.011
RMSE : 0.103
r2 score : 0.989


In [178]:
# val set의 예측결과
lgbm_val_mae = mean_absolute_error(y_val, pred_val)
lgbm_val_mse = mean_squared_error(y_val, pred_val)
lgbm_val_rmse = np.sqrt(lgbm_val_mse)
lgbm_val_r2 = r2_score(y_val, pred_val)
print('MAE : ', round(lgbm_val_mae, 3))
print('MSE : ', round(lgbm_val_mse, 3))
print('RMSE :', round(lgbm_val_rmse, 3))
print('r2 score :', round(lgbm_val_r2, 3))

MAE :  0.075
MSE :  0.024
RMSE : 0.155
r2 score : 0.976


In [179]:
# 최종 test set의 결과
lgbm_mae = mean_absolute_error(pred_test, y_test)
lgbm_mse = mean_squared_error(pred_test, y_test)
lgbm_rmse = np.sqrt(lgbm_mse)
lgbm_r2 = r2_score(pred_test, y_test)
print('MAE : ', round(lgbm_mae, 3))
print('MSE : ', round(lgbm_mse, 3))
print('RMSE :', round(lgbm_rmse, 3))
print('r2 score :', round(lgbm_r2, 3))

MAE :  0.074
MSE :  0.022
RMSE : 0.149
r2 score : 0.976


In [181]:
## 모델별 성능 비교
print('Model : Random Forest')
print('val :',  'RMSE: ', round(rf_val_rmse, 3), ',r2 score: ', round(rf_val_r2, 3))
print('Test :',  'RMSE: ', round(rf_rmse, 3), ',r2 score: ', round(rf_r2, 3))
print('------------------------------------')
print('Model : XGB')
print('val :',  'RMSE: ', round(reg_val_rmse, 3), ',r2 score: ', round(reg_val_r2, 3))
print('Test :',  'RMSE: ', round(reg_rmse, 3), ',r2 score: ', round(reg_r2, 3))
print('------------------------------------')
print('Model : LGBM')
print('val :',  'RMSE: ', round(lgbm_val_rmse, 3), ',r2 score: ', round(lgbm_val_r2, 3))
print('Test :',  'RMSE: ', round(lgbm_rmse, 3), ',r2 score: ', round(lgbm_r2, 3))

Model : Random Forest
val : RMSE:  0.151 ,r2 score:  0.977
Test : RMSE:  0.148 ,r2 score:  0.976
------------------------------------
Model : XGB
val : RMSE:  0.14 ,r2 score:  0.98
Test : RMSE:  0.131 ,r2 score:  0.982
------------------------------------
Model : LGBM
val : RMSE:  0.155 ,r2 score:  0.976
Test : RMSE:  0.149 ,r2 score:  0.976


## 미학습된 데이터에서 오차범위검증

In [81]:
check_list = []

In [82]:
def val(num):
    for i in range(num):
        randint = random.randint(0, len(a))
        if True in a.loc[randint].isnull()[2:].values:
            pass
        else:
            temp = a.loc[randint].values[2:]
            new=[]
            for i in temp:
                new.append(i)

            new_p = new.pop(6)
            new.insert(0, new_p)
            new = [new]
            new_scaled = scaler.transform(new)
            new_x , new_y = new_scaled[:, 1:], new_scaled[:, 0]
            new_pred = lgbm.predict(new_x)
            new_scaled[:,0] = new_pred
            pred = scaler.inverse_transform(new_scaled)
            check_list.append(round(abs(new[0][0] - pred[0][0]), 1))
#             print(new[0][0], '***', round(pred[0][0], 1), '***', round(abs(new[0][0] - pred[0][0]), 1))

In [83]:
a = pd.read_csv('data/보성/SURFACE_AGR_981_HR_2012_2012_2016.csv', encoding='cp949')
val(1000)

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

In [84]:
a = pd.read_csv('data/보성/SURFACE_AGR_981_HR_2013_2013_2016.csv', encoding='cp949')
val(1000)

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

In [85]:
a = pd.read_csv('data/보성/SURFACE_AGR_981_HR_2015_2015_2018.csv', encoding='cp949')
val(1000)

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

In [86]:
a = pd.read_csv('data/보성/SURFACE_AGR_981_HR_2016_2016_2018.csv', encoding='cp949')
val(1000)

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

In [87]:
a = pd.read_csv('data/오창가곡/SURFACE_AGR_977_HR_2018_2018_2019.csv', encoding='cp949')
val(1000)

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

In [88]:
a = pd.read_csv('data/춘천신북/SURFACE_AGR_978_HR_2012_2012_2016.csv', encoding='cp949')
val(1000)

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

In [89]:
a = pd.read_csv('data/화순능주/SURFACE_AGR_973_HR_2018_2018_2019.csv', encoding='cp949')
val(1000)

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have v

In [90]:
# 오차범위 평균, 무작위 검사 시행개수
np.mean(check_list), len(check_list)

(1.1835092828540226, 5494)

In [91]:
# 오차범위 최소, 최대
min(check_list), max(check_list)

(0.0, 12.4)

In [92]:
over_mean_list=[]
for i in check_list:
    if i > np.mean(check_list):
        over_mean_list.append(i)

In [93]:
# 오차범위 평균값 초과
len(over_mean_list)

1684

In [94]:
# 평균을 초과하는 값들의 최소, 최대
min(over_mean_list), max(over_mean_list)

(1.2, 12.4)

In [95]:
# 오차범위의 구간별 개수
Counter(sorted(over_mean_list))

Counter({1.2: 152,
         1.3: 129,
         1.4: 115,
         1.5: 110,
         1.6: 76,
         1.7: 66,
         1.8: 54,
         1.9: 58,
         2.0: 48,
         2.1: 47,
         2.2: 80,
         2.3: 62,
         2.4: 52,
         2.5: 40,
         2.6: 42,
         2.7: 36,
         2.8: 22,
         2.9: 37,
         3.0: 18,
         3.1: 15,
         3.2: 24,
         3.3: 11,
         3.4: 23,
         3.5: 17,
         3.6: 12,
         3.7: 15,
         3.8: 18,
         3.9: 11,
         4.0: 9,
         4.1: 9,
         4.2: 9,
         4.3: 6,
         4.4: 10,
         4.5: 9,
         4.6: 7,
         4.7: 6,
         4.8: 5,
         4.9: 7,
         5.0: 6,
         5.1: 5,
         5.2: 11,
         5.3: 9,
         5.4: 7,
         5.5: 6,
         5.6: 4,
         5.7: 11,
         5.8: 5,
         5.9: 8,
         6.1: 5,
         6.2: 4,
         6.3: 5,
         6.4: 4,
         6.5: 7,
         6.6: 6,
         6.7: 15,
         6.8: 17,
         6.